In [1]:
import numpy as np
import pandas as pd
#import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data_init = pd.read_excel("~/Projects/Sberbank/total4.xlsx")

In [2]:
data_init = pd.read_excel("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/total4.xlsx")

In [3]:
data = data_init.copy()

In [4]:
len(data[data.state > 0])

26447

In [5]:
data["price_m"] = data.price_doc / data.full_sq
#data["price_m"] = np.log1p(data.price_doc)

In [27]:
data["year"] = [int(d[:4]) for d in data.timestamp.values]

In [111]:
cian = pd.read_csv("~/Projects/Sberbank/cian_prices", sep=";", header=None,
                  names=["sub_area", "avp", "growth"])

In [112]:
cian.head()

,sub_area,avp,growth
0,Ajeroport,221 317,0.9
1,Akademicheskoe,213 748,0.4
2,Alekseevskoe,213 848,0.6
3,Altuf'evskoe,151 306,-1.3
4,Arbat,526 173,7.2


In [113]:
cian.avp = cian.avp.apply(lambda x: float(str(x).replace(" ", "")))

In [114]:
cian["av"] = cian.avp

In [6]:
data["month"] = [(d[:4])+(d[5:7]) for d in data.timestamp.values]

In [7]:
#avp = pd.DataFrame()
#avp["sub_area"] = np.unique(data.sub_area.values)
#avp["av_p"] = [data[(data.price_doc != 0)&(data.sub_area == r)].price_m.mean() 
#               for r in avp.sub_area.values]
#data = data.merge(avp, on="sub_area")
#data["low"] = np.where(data.price_m.values / data.av_p.values < 0.4, 1, 0)
#data["low"] = np.where(data.price_doc.values == 0, 0, data.low.values)
#data = data[data.low == 0].reset_index(drop=True)
#data.drop(["low", "av_p"], 1, inplace=True)

In [8]:
data["expl"] = -data.build_year + data.year

AttributeError: 'DataFrame' object has no attribute 'year'

In [9]:
ms = np.random.choice(np.unique(data[data.price_doc != 0].month.values)[18:], 
                      size = 6, replace=False)
ms
ms = ['201501', '201502', '201503', '201504', '201505', '201506']

In [10]:
data["val"] = [1 if m in ms else 0 for m in data.month.values]

In [11]:
#data = data[((data.val == 0)&((data.price_m > 10000)|(data.price_m == 0)))|(data.val == 1)]

In [12]:
#data.drop("av", 1, inplace=True)
av = pd.DataFrame()
av["sub_area"] = np.unique(data.sub_area.values)
#av["av"] = [data[(data.price_doc != 0)&(data.val == 0)&(data.sub_area == s)].price_m.mean() 
#            for s in av.sub_area.values]
av["av"] = [data[(data.price_doc != 0)&(data.sub_area == s)].price_m.mean() 
            for s in av.sub_area.values]

data = data.merge(av, on="sub_area")

In [120]:
data = data.merge(cian[["sub_area", "av"]], on="sub_area")

In [121]:
months = list(np.unique(data.month.values))
metr = [1]*16 + [128900,129600,131900,132100,127800,124550,125940,124860,130225,130800,134470,
               133000,135310,140700,143680,150535,139000,138100,140400,142600,145890,145100,
               146360,151040,147250,152600,154200,157960,159430,159880,160740,159100,156600,
               153385,151970,150060,147890,150670,147500,144768,143445,141200]

av_prs = []

for i,m in enumerate(months):
    if i < 47:
        av_prs.append(float(data[(data.month == m)].price_m.mean()))
    else:
        av_prs.append(av_prs[i-1]*metr[i]/metr[i-1])

In [122]:
#pv = []
#
#for i in range(len(data)):
#    ind = months.index(data.month.values[i])
#    if data.month.values[i] in ms:
#        pv.append(av_prs[ind-1]*metr[ind]/metr[ind-1])
#    else:
#        pv.append(av_prs[ind])
#data["pv"] = pv

In [123]:
#For final forecast

pv = []

for i in range(len(data)):
    ind = months.index(data.month.values[i])
    pv.append(av_prs[ind])

data["pv"] = pv

In [124]:
#data.drop(["id", "timestamp", "sub_area", "month"],1,inplace=True)
data.drop(["timestamp", "month"],1,inplace=True)

In [125]:
for col in data.columns:
    if data.ix[:, col].dtype == 'object':
        data.ix[:, col] = data.ix[:, col].apply(lambda x: str(x))
        le = LabelEncoder()
        data.ix[:, col] = le.fit_transform(data.ix[:, col].values)

In [126]:
#data["test_feature"] = data.floor / data.max_floor

In [127]:
to_del2 = ["thermal_power_plant_raion", "incineration_raion", "oil_chemistry_raion",
         "radiation_raion", "railroad_terminal_raion", "nuclear_reactor_raion", 
          "detention_facility_raion", "metro_min_walk", "children_school", "children_preschool",
         "7_14_all", "0_6_all", "0_13_all"]

In [128]:
#cats = ["ID_big_road1","ID_metro","ID_big_road2", "ID_railroad_terminal", 
#        "ID_bus_terminal", "ID_railroad_station_avto","ID_railroad_station_walk"]

In [130]:
data.drop(to_del2, 1, inplace=True)

In [131]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [132]:
#data = data[data.year >= 2012]

In [133]:
df = data.drop(["price_m", "price_doc", "id"],1)

In [134]:
%%time

ind = data[(data.price_doc != 0)&(data.val == 1)].index
nind = list(set(data[data.price_m != 0].index) - set(ind))

df.drop("val", 1, inplace=True)

y_val, y_train = data.loc[ind].price_m.values, data.loc[nind].price_m.values
X_val, X_train = df.loc[ind].values, df.loc[nind].values

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df.columns)
dval = xgb.DMatrix(X_val, y_val, feature_names=df.columns)

partial_model = xgb.train(xgb_params, dtrain, num_boost_round=600, evals=[(dval, 'val')],
                       early_stopping_rounds=20, verbose_eval=20)

preds = partial_model.predict(dval)
#preds = np.exp(preds) - 1
preds = X_val[:,0] * preds
real = data.loc[ind].price_doc.values
l = (np.log1p(preds) - np.log1p(real))**2
print(np.sqrt((1/len(preds))*sum(l)))

[0]	val-rmse:146642
Will train until val-rmse hasn't improved in 20 rounds.
[20]	val-rmse:68978
[40]	val-rmse:46823.8
[60]	val-rmse:41394.6
[80]	val-rmse:39902.4
[100]	val-rmse:39408.2
[120]	val-rmse:39106.9
[140]	val-rmse:38958.6
[160]	val-rmse:38874
[180]	val-rmse:38852.3
[200]	val-rmse:38761.8
[220]	val-rmse:38678.3
[240]	val-rmse:38614.3
[260]	val-rmse:38614.7
[280]	val-rmse:38597.6
[300]	val-rmse:38590.8
[320]	val-rmse:38560.3
[340]	val-rmse:38559.3
Stopping. Best iteration:
[329]	val-rmse:38533

0.388465683055
CPU times: user 58.7 s, sys: 1.17 s, total: 59.9 s
Wall time: 19.6 s


In [ ]:
0.389337579385

In [23]:
real = data.loc[ind]
real["preds"] = preds
real["delta"] = real.price_doc / real.preds - 1
real.sort(columns="delta", inplace=True, ascending=0)
#real.drop("sub_area",1,inplace=True)

/Users/daber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [24]:
real = real.merge(data_init[["id", "sub_area"]], on="id")

In [25]:
temp = real.ix[:,["id","year", "full_sq", "life_sq", "floor", "max_floor", 
           "build_year", "sub_area", "price_doc", "preds", "delta"]]

In [27]:
temp[temp.delta > 0.3]

,id,year,full_sq,life_sq,floor,max_floor,build_year,sub_area,price_doc,preds,delta
0,27463,2015,151.0,NaN,15.0,NaN,2015.0,Ajeroport,71249624,2.700856e+07,1.638039
1,30047,2015,61.0,NaN,2.0,25.0,2016.0,Tverskoe,12505200,5.600120e+06,1.233023
2,27881,2015,121.0,74.0,6.0,7.0,2006.0,Kuncevo,44600000,2.054544e+07,1.170799
3,29378,2015,150.0,79.0,4.0,18.0,2004.0,Kuncevo,52000000,2.573956e+07,1.020236
4,30444,2015,113.0,NaN,26.0,26.0,2015.0,Tverskoe,25039300,1.269884e+07,0.971779
5,30028,2015,108.0,NaN,8.0,16.0,2015.0,Alekseevskoe,34645000,1.759097e+07,0.969476
6,27566,2015,30.0,18.0,1.0,5.0,1960.0,Presnenskoe,6300000,3.263558e+06,0.930408
7,30437,2015,76.0,44.0,6.0,12.0,2006.0,Donskoe,17200000,9.234664e+06,0.862547
8,27645,2015,39.0,19.0,11.0,17.0,2008.0,Poselenie Moskovskij,6300000,3.390049e+06,0.858380
9,28434,2015,47.0,31.0,1.0,12.0,1965.0,Kuz'minki,12000000,6.536195e+06,0.835930


In [226]:
temp[temp.sub_area == "Kuncevo"]

,id,year,full_sq,life_sq,floor,max_floor,build_year,sub_area,price_doc,preds,delta
3,27881,2015,121.0,74.0,6.0,7.0,2006.0,Kuncevo,44600000,2.171051e+07,1.054305
5,29378,2015,150.0,79.0,4.0,18.0,2004.0,Kuncevo,52000000,2.795967e+07,0.859822
14,29884,2015,72.0,44.0,3.0,14.0,2006.0,Kuncevo,18000000,1.061955e+07,0.694987
17,29913,2015,127.0,81.0,12.0,22.0,1999.0,Kuncevo,38900000,2.393234e+07,0.625415
20,29208,2015,74.0,42.0,17.0,17.0,2010.0,Kuncevo,16200000,1.008785e+07,0.605892
54,29462,2015,60.0,44.0,2.0,9.0,1968.0,Kuncevo,10500000,7.194197e+06,0.459510
269,30024,2015,38.0,23.0,4.0,9.0,1964.0,Kuncevo,8000000,6.208998e+06,0.288453
430,30287,2015,38.0,19.0,17.0,17.0,2008.0,Kuncevo,8400000,6.801765e+06,0.234974
878,27467,2015,37.0,17.0,7.0,17.0,2006.0,Kuncevo,7300000,6.342620e+06,0.150944
997,27597,2015,38.0,18.0,11.0,16.0,1987.0,Kuncevo,7800000,6.868873e+06,0.135557


In [36]:
output = pd.read_csv("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/output.csv")
output = output[1:-1]

In [37]:
output.head()

,id,price_doc
1,30474,5.589361e+06
2,30475,8.472448e+06
3,30476,5.553109e+06
4,30477,5.822925e+06
5,30478,5.079535e+06


In [16]:
output.id = output.id.astype(str)
output.price_doc = output.price_doc.astype(str)
output = output[output.price_doc.str.find("==") == -1]
output = output[output.id.str.find("==") == -1]

In [18]:
output.id = output.id.apply(lambda x: int(x))
data.id = data.id.apply(lambda x: int(x))
output.price_doc = output.price_doc.apply(lambda x: float(x))
data.price_doc = data.price_doc.apply(lambda x: float(x))

In [19]:
data = data.merge(output, on="id", how="outer")

In [20]:
data.price_doc_y.fillna(0, inplace=True)

In [21]:
data["price_m2"] = data.price_doc_y / data.full_sq

In [33]:
temp = data[["id", "full_sq", "build_year", "max_floor", "price_m", "price_m2"]][data.sub_area == "Ajeroport"]

In [31]:
temp[temp.price_m2 > 0].price_m2.mean()

191207.7737504852

In [30]:
temp[temp.price_m > 50000].price_m.mean()

186647.68755205264

In [34]:
temp[temp.price_m2 > 0]

,id,full_sq,build_year,max_floor,price_m,price_m2
25397,30621,51.0,1967.0,9.0,0.0,178275.140625
25398,30621,51.0,1967.0,9.0,0.0,226221.422106
25399,30790,41.9,1963.0,5.0,0.0,166344.796875
25400,30790,41.9,1963.0,5.0,0.0,192449.043682
25401,31751,45.7,1960.0,5.0,0.0,181150.125000
25402,31751,45.7,1960.0,5.0,0.0,195568.536822
25403,32405,72.5,2006.0,22.0,0.0,200296.531250
25404,32405,72.5,2006.0,22.0,0.0,244875.733520
25405,32851,52.3,1960.0,5.0,0.0,200646.515625
25406,32851,52.3,1960.0,5.0,0.0,252850.603451


In [135]:
X = data[data.price_doc != 0].drop(["price_m", "price_doc", "val", "id"],1)
y = data[data.price_doc != 0].price_m
X1 = data[data.price_doc == 0].drop(["price_m", "price_doc", "val", "id"],1)
X10 = X.values
X11 = X1.values
dtrain = xgb.DMatrix(X, y, feature_names=X1.columns)
dtest = xgb.DMatrix(X1)
ids = data[data.price_doc == 0].id.values

In [136]:
model = xgb.train(xgb_params, dtrain, num_boost_round=329)

In [137]:
preds = model.predict(dtest)
#preds = np.exp(preds) - 1
preds = X11[:,0] * preds

In [138]:
preds

array([ 5081273.503125 ,  6148572.828125 ,  8248727.2890625, ...,
        4010857.64375  ,  3432439.753125 ,  3089074.3      ])

In [139]:
output = pd.DataFrame({"id": ids, "price_doc": preds})
output.sort(columns="id", inplace=True)
output.to_csv("~/Projects/Sberbank/output.csv", index=False)

/Users/daber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [140]:
output.head()

,id,price_doc
2407,30474,5.589361e+06
6929,30475,8.472448e+06
4610,30476,5.553109e+06
6727,30477,5.822925e+06
6930,30478,5.079535e+06


In [38]:
res = list(model.get_fscore().items())
res.sort(key=lambda x: x[1], reverse=True)
res

[('full_sq', 830),
 ('life_sq', 359),
 ('floor', 357),
 ('pv', 248),
 ('max_floor', 247),
 ('build_year', 230),
 ('year', 140),
 ('kindergarten_km', 134),
 ('kitch_sq', 128),
 ('av', 122),
 ('public_transport_station_km', 122),
 ('metro_min_avto', 122),
 ('state', 115),
 ('green_zone_km', 94),
 ('public_healthcare_km', 94),
 ('catering_km', 87),
 ('expl', 86),
 ('park_km', 85),
 ('railroad_km', 84),
 ('cemetery_km', 82),
 ('metro_km_avto', 80),
 ('fitness_km', 78),
 ('additional_education_km', 78),
 ('metro_km_walk', 78),
 ('workplaces_km', 77),
 ('hospice_morgue_km', 76),
 ('industrial_km', 75),
 ('market_shop_km', 73),
 ('material', 72),
 ('school_km', 69),
 ('shopping_centers_km', 69),
 ('big_road1_km', 69),
 ('mosque_km', 66),
 ('theater_km', 66),
 ('num_room', 65),
 ('water_km', 65),
 ('university_km', 63),
 ('railroad_station_walk_km', 60),
 ('swim_pool_km', 60),
 ('church_synagogue_km', 58),
 ('office_km', 57),
 ('big_market_km', 56),
 ('green_part_1500', 55),
 ('big_church_km',

In [40]:
r = pd.DataFrame()
col, imp = [], []
for i in range(len(res)):
    col.append(res[i][0])
    imp.append(res[i][1])
r["col"] = col
r["imp"] = imp
r.sort(columns="imp", inplace=True)

/Users/daber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [41]:
r.head()

,col,imp
261,work_all,1
251,7_14_female,1
252,big_church_count_1000,1
253,0_6_male,1
254,work_male,1


In [106]:
to_del = r[r.imp <= 50].col.values

In [63]:
preds0 = model.predict(dtrain)

In [64]:
data["preds"] = list(preds0) + list(preds)

In [47]:
to_take = ["id", "full_sq", "life_sq", "floor", "max_floor", "build_year", "num_room",
          "state", 'kitch_sq','metro_min_avto',"price_m"]

In [48]:
temp = data[to_take]

In [49]:
temp = temp.merge(data_init[["id", "sub_area"]], on="id")

In [50]:
temp.drop(["metro_min_avto", "num_room"],1,inplace=True)

In [51]:
temp[(temp.sub_area == "Brateevo")&(temp.build_year > 0)]

,id,full_sq,life_sq,floor,max_floor,build_year,state,kitch_sq,price_m,sub_area
10467,8579,58.0,31.0,17.0,17.0,1985.0,3.0,10.0,146551.724138,Brateevo
10473,9142,39.0,19.0,12.0,17.0,1996.0,3.0,8.0,153846.153846,Brateevo
10477,10216,39.0,18.0,6.0,25.0,2001.0,3.0,9.0,153846.153846,Brateevo
10478,10358,38.0,20.0,1.0,14.0,1987.0,2.0,8.0,55263.157895,Brateevo
10479,10406,38.0,20.0,8.0,22.0,1985.0,3.0,10.0,139473.684211,Brateevo
10480,10452,40.0,19.0,6.0,25.0,2000.0,3.0,9.0,162500.000000,Brateevo
10481,10859,60.0,34.0,2.0,17.0,1989.0,2.0,10.0,129166.666667,Brateevo
10482,11017,73.0,NaN,4.0,17.0,1996.0,3.0,10.0,68493.150685,Brateevo
10483,11049,52.0,31.0,13.0,17.0,1997.0,NaN,8.0,142307.692308,Brateevo
10484,11203,39.0,20.0,4.0,17.0,1986.0,2.0,8.0,51282.051282,Brateevo


In [52]:
np.unique(temp.sub_area)

array(['Ajeroport', 'Akademicheskoe', 'Alekseevskoe', "Altuf'evskoe",
       'Arbat', 'Babushkinskoe', 'Basmannoe', 'Begovoe', 'Beskudnikovskoe',
       'Bibirevo', 'Birjulevo Vostochnoe', 'Birjulevo Zapadnoe',
       'Bogorodskoe', 'Brateevo', 'Butyrskoe', 'Caricyno', 'Cheremushki',
       "Chertanovo Central'noe", 'Chertanovo Juzhnoe',
       'Chertanovo Severnoe', 'Danilovskoe', 'Dmitrovskoe', 'Donskoe',
       'Dorogomilovo', 'Filevskij Park', 'Fili Davydkovo', 'Gagarinskoe',
       "Gol'janovo", 'Golovinskoe', 'Hamovniki', 'Horoshevo-Mnevniki',
       'Horoshevskoe', 'Hovrino', 'Ivanovskoe', 'Izmajlovo', 'Jakimanka',
       'Jaroslavskoe', 'Jasenevo', 'Juzhnoe Butovo', 'Juzhnoe Medvedkovo',
       'Juzhnoe Tushino', 'Juzhnoportovoe', 'Kapotnja', "Kon'kovo",
       'Koptevo', 'Kosino-Uhtomskoe', 'Kotlovka', "Krasnosel'skoe",
       'Krjukovo', 'Krylatskoe', 'Kuncevo', 'Kurkino', "Kuz'minki",
       'Lefortovo', 'Levoberezhnoe', 'Lianozovo', 'Ljublino',
       'Lomonosovskoe', 'Losi

In [40]:
data.head()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,sport_count_5000,market_count_5000,price_doc,price_m,year,expl,val,av,pv,preds
0,1,43.0,27.0,4.0,9.0,1.0,1974.0,2.0,6.0,2.0,...,52,4,5850000,136046.511628,2011,37.0,0,137143.245469,148358.413133,5.717777e+06
1,8,44.0,NaN,5.0,9.0,1.0,1974.0,2.0,6.0,NaN,...,47,4,2000000,45454.545455,2011,37.0,0,137143.245469,131932.175649,4.671511e+06
2,33,38.0,34.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,43,5,4900000,128947.368421,2011,NaN,0,137143.245469,131932.175649,4.295615e+06
3,71,32.0,19.0,9.0,9.0,1.0,1975.0,1.0,6.0,2.0,...,55,5,5300000,165625.000000,2011,36.0,0,137143.245469,118790.067943,4.502550e+06
4,103,44.0,28.0,4.0,9.0,1.0,1974.0,2.0,6.0,NaN,...,47,4,5950000,135227.272727,2011,37.0,0,137143.245469,118790.067943,5.184796e+06


In [7]:
to_take = ["id", "full_sq", "life_sq", "floor", "max_floor", "build_year", "num_room",
          "state", "sub_area", 'kitch_sq','metro_min_avto',"price_doc", "price_m"]

In [8]:
df = data[to_take]

In [9]:
df[df.build_year > 0]

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_avto,price_doc,price_m
0,1,43.00,27.00,4.0,9.0,1974.0,2.0,2.5,Bibirevo,6.00,2.590241,5850000,136046.511628
7,8,44.00,44.00,5.0,9.0,1974.0,2.0,NaN,Bibirevo,6.00,3.623270,2000000,45454.545455
70,71,32.00,19.00,9.0,9.0,1975.0,1.0,2.5,Bibirevo,6.00,3.914421,5300000,165625.000000
102,103,44.00,28.00,4.0,9.0,1974.0,2.0,NaN,Bibirevo,6.00,3.739761,5950000,135227.272727
262,263,102.00,66.00,4.0,NaN,1999.0,NaN,NaN,Ajeroport,NaN,2.586275,17300000,169607.843137
298,299,43.00,28.00,5.0,NaN,1959.0,NaN,NaN,Ajeroport,NaN,3.270153,8750000,203488.372093
337,338,42.00,27.00,3.0,NaN,1963.0,2.0,2.5,Akademicheskoe,5.00,1.941803,8750000,208333.333333
366,367,63.00,45.00,2.0,12.0,1975.0,3.0,2.5,Bibirevo,6.00,3.444894,7600000,120634.920635
501,502,32.00,18.00,1.0,9.0,1979.0,1.0,2.5,Akademicheskoe,6.00,3.740331,5300000,165625.000000
556,557,39.00,21.00,9.0,12.0,1968.0,2.0,2.0,Savelki,7.00,28.428549,4300000,110256.410256


In [10]:
df = df[df.sub_area == "Kuncevo"].reset_index()

In [104]:
df[df.id == 27672]

,index,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_avto,price_doc,price_m
173,27669,27672,96.0,62.0,10.0,22.0,2000.0,3.0,4.0,Kuncevo,11.0,2.461086,15000000,156250.0


In [12]:
df2 = df.copy()
df2.fillna(0, inplace=True)
df2 = df2[df2.price_doc > 0]
df2 = df2.drop(["id", "sub_area", "price_doc", "price_m", 'metro_min_avto'],1)

In [13]:
from sklearn.preprocessing import StandardScaler
import scipy as sc

In [14]:
ss = StandardScaler()
df3 = ss.fit_transform(df2)

In [105]:
v = df3[173]
id_best, dist = 0, sc.linalg.norm(df3[0]-v)
for i in range(len(df3)):
    if i != 173 and i != 156 and i != 159:
        if sc.linalg.norm(df3[i]-v) < dist:
            dist = sc.linalg.norm(df3[i]-v)
            id_best = i

In [106]:
id_best

152

In [112]:
df.loc[173]

index                27669
id                   27672
full_sq                 96
life_sq                 62
floor                   10
max_floor               22
build_year            2000
num_room                 3
state                    4
sub_area           Kuncevo
kitch_sq                11
metro_min_avto     2.46109
price_doc         15000000
price_m             156250
Name: 173, dtype: object

In [140]:
data[data.id == 11106]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,price_m
11103,11106,2013-10-09,114.0,67.0,6.0,17.0,2.0,2012.0,3.0,16.0,...,12,1,8,18,0,2,48,0,14700000,128947.368421


In [111]:
214/170-1

0.2588235294117647